In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
import requests
import json
from datetime import datetime
import os

class CurrencyConverter:
    def __init__(self, root):
        self.root = root
        self.root.title("Global Currency Converter")
        self.root.geometry("800x600")
        self.root.resizable(False, False)
        
        # Color scheme
        self.bg_color = "#f0f0f0"
        self.primary_color = "#2563eb"
        self.secondary_color = "#1e40af"
        self.text_color = "#1f2937"
        
        self.root.configure(bg=self.bg_color)
        
        # Exchange rates data
        self.exchange_rates = {}
        self.base_currency = "USD"
        self.last_update = None
        self.cache_file = "exchange_rates_cache.json"
        
        # Favorites
        self.favorites = []
        self.favorites_file = "favorites.json"
        
        # Load cached data
        self.load_cache()
        self.load_favorites()
        
        # Create UI
        self.create_widgets()
        
        # Fetch exchange rates
        self.update_exchange_rates()
    
    def create_widgets(self):
        # Title
        title_frame = tk.Frame(self.root, bg=self.primary_color, height=60)
        title_frame.pack(fill=tk.X)
        title_frame.pack_propagate(False)
        
        title_label = tk.Label(
            title_frame, 
            text="🌍 Global Currency Converter",
            font=("Arial", 20, "bold"),
            bg=self.primary_color,
            fg="white"
        )
        title_label.pack(pady=15)
        
        # Main container
        main_frame = tk.Frame(self.root, bg=self.bg_color)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        # Conversion Frame
        conversion_frame = tk.LabelFrame(
            main_frame, 
            text="Currency Conversion",
            font=("Arial", 12, "bold"),
            bg=self.bg_color,
            fg=self.text_color,
            padx=20,
            pady=20
        )
        conversion_frame.pack(fill=tk.BOTH, expand=True)
        
        # Amount input
        amount_frame = tk.Frame(conversion_frame, bg=self.bg_color)
        amount_frame.pack(fill=tk.X, pady=10)
        
        tk.Label(
            amount_frame, 
            text="Amount:",
            font=("Arial", 11),
            bg=self.bg_color,
            fg=self.text_color
        ).pack(side=tk.LEFT, padx=(0, 10))
        
        self.amount_entry = tk.Entry(
            amount_frame,
            font=("Arial", 12),
            width=20,
            relief=tk.SOLID,
            borderwidth=1
        )
        self.amount_entry.pack(side=tk.LEFT)
        self.amount_entry.insert(0, "1.00")
        self.amount_entry.bind("<KeyRelease>", lambda e: self.convert_currency())
        
        # From Currency
        from_frame = tk.Frame(conversion_frame, bg=self.bg_color)
        from_frame.pack(fill=tk.X, pady=10)
        
        tk.Label(
            from_frame,
            text="From:",
            font=("Arial", 11),
            bg=self.bg_color,
            fg=self.text_color,
            width=10,
            anchor="w"
        ).pack(side=tk.LEFT)
        
        self.from_currency = ttk.Combobox(
            from_frame,
            font=("Arial", 11),
            width=30,
            state="readonly"
        )
        self.from_currency.pack(side=tk.LEFT, padx=(0, 10))
        self.from_currency.bind("<<ComboboxSelected>>", lambda e: self.convert_currency())
        
        self.add_favorite_from_btn = tk.Button(
            from_frame,
            text="⭐",
            font=("Arial", 10),
            command=self.add_to_favorites_from,
            bg="white",
            relief=tk.SOLID,
            borderwidth=1,
            width=3
        )
        self.add_favorite_from_btn.pack(side=tk.LEFT)
        
        # Swap button
        swap_frame = tk.Frame(conversion_frame, bg=self.bg_color)
        swap_frame.pack(pady=10)
        
        swap_btn = tk.Button(
            swap_frame,
            text="⇅ Swap",
            font=("Arial", 11, "bold"),
            command=self.swap_currencies,
            bg=self.primary_color,
            fg="white",
            relief=tk.FLAT,
            padx=20,
            pady=5,
            cursor="hand2"
        )
        swap_btn.pack()
        
        # To Currency
        to_frame = tk.Frame(conversion_frame, bg=self.bg_color)
        to_frame.pack(fill=tk.X, pady=10)
        
        tk.Label(
            to_frame,
            text="To:",
            font=("Arial", 11),
            bg=self.bg_color,
            fg=self.text_color,
            width=10,
            anchor="w"
        ).pack(side=tk.LEFT)
        
        self.to_currency = ttk.Combobox(
            to_frame,
            font=("Arial", 11),
            width=30,
            state="readonly"
        )
        self.to_currency.pack(side=tk.LEFT, padx=(0, 10))
        self.to_currency.bind("<<ComboboxSelected>>", lambda e: self.convert_currency())
        
        self.add_favorite_to_btn = tk.Button(
            to_frame,
            text="⭐",
            font=("Arial", 10),
            command=self.add_to_favorites_to,
            bg="white",
            relief=tk.SOLID,
            borderwidth=1,
            width=3
        )
        self.add_favorite_to_btn.pack(side=tk.LEFT)
        
        # Result Display
        result_frame = tk.Frame(conversion_frame, bg="white", relief=tk.SOLID, borderwidth=1)
        result_frame.pack(fill=tk.X, pady=20, ipady=20)
        
        self.result_label = tk.Label(
            result_frame,
            text="Enter amount and select currencies",
            font=("Arial", 14, "bold"),
            bg="white",
            fg=self.primary_color
        )
        self.result_label.pack()
        
        # Exchange Rate Info
        self.rate_info_label = tk.Label(
            conversion_frame,
            text="",
            font=("Arial", 9),
            bg=self.bg_color,
            fg="#6b7280"
        )
        self.rate_info_label.pack()
        
        # Bottom Frame
        bottom_frame = tk.Frame(self.root, bg=self.bg_color)
        bottom_frame.pack(fill=tk.X, padx=20, pady=(0, 20))
        
        # Update button
        update_btn = tk.Button(
            bottom_frame,
            text="🔄 Update Rates",
            font=("Arial", 10),
            command=self.update_exchange_rates,
            bg=self.secondary_color,
            fg="white",
            relief=tk.FLAT,
            padx=15,
            pady=5,
            cursor="hand2"
        )
        update_btn.pack(side=tk.LEFT, padx=(0, 10))
        
        # Favorites button
        favorites_btn = tk.Button(
            bottom_frame,
            text="⭐ View Favorites",
            font=("Arial", 10),
            command=self.show_favorites,
            bg="white",
            fg=self.text_color,
            relief=tk.SOLID,
            borderwidth=1,
            padx=15,
            pady=5,
            cursor="hand2"
        )
        favorites_btn.pack(side=tk.LEFT)
        
        # Last update info
        self.update_info_label = tk.Label(
            bottom_frame,
            text="",
            font=("Arial", 9),
            bg=self.bg_color,
            fg="#6b7280"
        )
        self.update_info_label.pack(side=tk.RIGHT)
    
    def update_exchange_rates(self):
        try:
            # Using exchangerate-api.com (free tier available)
            url = f"https://api.exchangerate-api.com/v4/latest/{self.base_currency}"
            response = requests.get(url, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                self.exchange_rates = data['rates']
                self.last_update = datetime.now()
                
                # Save to cache
                self.save_cache()
                
                # Update currency lists
                self.update_currency_lists()
                
                # Update UI
                self.update_info_label.config(
                    text=f"Last updated: {self.last_update.strftime('%Y-%m-%d %H:%M:%S')}"
                )
                
                messagebox.showinfo("Success", "Exchange rates updated successfully!")
                
                # Convert if currencies are selected
                self.convert_currency()
            else:
                messagebox.showerror("Error", "Failed to fetch exchange rates. Using cached data.")
        except requests.RequestException as e:
            messagebox.showerror("Error", f"Network error: {str(e)}\nUsing cached data if available.")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")
    
    def update_currency_lists(self):
        currencies = sorted(list(self.exchange_rates.keys()))
        
        # Get current selections
        current_from = self.from_currency.get()
        current_to = self.to_currency.get()
        
        # Update comboboxes
        self.from_currency['values'] = currencies
        self.to_currency['values'] = currencies
        
        # Set default or restore selections
        if current_from in currencies:
            self.from_currency.set(current_from)
        else:
            self.from_currency.set("USD" if "USD" in currencies else currencies[0])
        
        if current_to in currencies:
            self.to_currency.set(current_to)
        else:
            self.to_currency.set("EUR" if "EUR" in currencies else currencies[1] if len(currencies) > 1 else currencies[0])
    
    def convert_currency(self):
        try:
            amount = float(self.amount_entry.get())
            from_curr = self.from_currency.get()
            to_curr = self.to_currency.get()
            
            if not from_curr or not to_curr:
                return
            
            if from_curr not in self.exchange_rates or to_curr not in self.exchange_rates:
                return
            
            # Convert through USD base
            amount_in_usd = amount / self.exchange_rates[from_curr]
            converted_amount = amount_in_usd * self.exchange_rates[to_curr]
            
            # Update result
            self.result_label.config(
                text=f"{amount:,.2f} {from_curr} = {converted_amount:,.2f} {to_curr}"
            )
            
            # Update rate info
            rate = self.exchange_rates[to_curr] / self.exchange_rates[from_curr]
            self.rate_info_label.config(
                text=f"1 {from_curr} = {rate:.4f} {to_curr}"
            )
            
        except ValueError:
            self.result_label.config(text="Please enter a valid number")
        except Exception as e:
            self.result_label.config(text=f"Error: {str(e)}")
    
    def swap_currencies(self):
        from_curr = self.from_currency.get()
        to_curr = self.to_currency.get()
        
        self.from_currency.set(to_curr)
        self.to_currency.set(from_curr)
        
        self.convert_currency()
    
    def add_to_favorites_from(self):
        currency = self.from_currency.get()
        self.add_to_favorites(currency)
    
    def add_to_favorites_to(self):
        currency = self.to_currency.get()
        self.add_to_favorites(currency)
    
    def add_to_favorites(self, currency):
        if currency and currency not in self.favorites:
            self.favorites.append(currency)
            self.save_favorites()
            messagebox.showinfo("Success", f"{currency} added to favorites!")
        elif currency in self.favorites:
            messagebox.showinfo("Info", f"{currency} is already in favorites!")
    
    def show_favorites(self):
        if not self.favorites:
            messagebox.showinfo("Favorites", "No favorite currencies yet!")
            return
        
        # Create favorites window
        fav_window = tk.Toplevel(self.root)
        fav_window.title("Favorite Currencies")
        fav_window.geometry("400x300")
        fav_window.configure(bg=self.bg_color)
        
        tk.Label(
            fav_window,
            text="Your Favorite Currencies",
            font=("Arial", 14, "bold"),
            bg=self.bg_color,
            fg=self.text_color
        ).pack(pady=10)
        
        # Listbox for favorites
        listbox_frame = tk.Frame(fav_window, bg=self.bg_color)
        listbox_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        scrollbar = tk.Scrollbar(listbox_frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        listbox = tk.Listbox(
            listbox_frame,
            font=("Arial", 11),
            yscrollcommand=scrollbar.set
        )
        listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.config(command=listbox.yview)
        
        for fav in self.favorites:
            listbox.insert(tk.END, fav)
        
        # Remove button
        def remove_favorite():
            selection = listbox.curselection()
            if selection:
                currency = listbox.get(selection[0])
                self.favorites.remove(currency)
                self.save_favorites()
                listbox.delete(selection[0])
                messagebox.showinfo("Success", f"{currency} removed from favorites!")
        
        remove_btn = tk.Button(
            fav_window,
            text="Remove Selected",
            font=("Arial", 10),
            command=remove_favorite,
            bg="#dc2626",
            fg="white",
            relief=tk.FLAT,
            padx=15,
            pady=5
        )
        remove_btn.pack(pady=10)
    
    def save_cache(self):
        cache_data = {
            'rates': self.exchange_rates,
            'timestamp': self.last_update.isoformat() if self.last_update else None
        }
        with open(self.cache_file, 'w') as f:
            json.dump(cache_data, f)
    
    def load_cache(self):
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r') as f:
                    cache_data = json.load(f)
                    self.exchange_rates = cache_data.get('rates', {})
                    timestamp = cache_data.get('timestamp')
                    if timestamp:
                        self.last_update = datetime.fromisoformat(timestamp)
            except Exception as e:
                print(f"Error loading cache: {e}")
    
    def save_favorites(self):
        with open(self.favorites_file, 'w') as f:
            json.dump(self.favorites, f)
    
    def load_favorites(self):
        if os.path.exists(self.favorites_file):
            try:
                with open(self.favorites_file, 'r') as f:
                    self.favorites = json.load(f)
            except Exception as e:
                print(f"Error loading favorites: {e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = CurrencyConverter(root)
    root.mainloop()
    